# Import requirements

transformers 및 wandb 패키지 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 15.1 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 596 kB 70.5 MB/s 
     |████████████████████████████████| 6.6 MB 56.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 16.0 MB/s 
     |████████████████████████████████| 147 kB 71.3 MB/s 
     |████████████████████████████████| 181 kB 57.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=700fe11839cc6b4d2fabd22e4a19ef92b02b33aa975e966821b278d1cd3fcd7f
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


#wandb login & init

위에서 다운받은 wandb 패키지를 import하고 wandb.login() 메소드를 통해 wandb에 로그인합니다. 

wandb 사이트에서 API Key를 복사해 붙혀 넣거나 colab과 같은 브라우저에서  wandb에 로그인해있는 경우

 별도로 API key 입력 없이 로그인이 진행됩니다.

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

https://docs.wandb.ai/ref/python/init에서 init 메소드에 사용 가능한 parameter들에 대한 설명을 자세하게 확인할 수 있습니다.

주로 사용하는 parameter들을 정리해보았습니다.



*   project (str, optional) : 새로운 run을 보낼 프로젝트의 이름입니다. 프로젝트가 지정되지 않은 경우 "uncategorized"프로젝트로 보내집니다.
*   entity (str, optioanl) : run을 보낼 유저이름 또는 팀의 이름입니다. run이 저장소로 보내지기 전에 해당 entity가 존재해야하므로 해당 entity가 없다면 run을 시작하기 전 UI에서 계정이나 팀을 만들어야 합니다.

*   config (dict, argparse, absl.flag, str, optional) : wnadb의 config 모델에 들어갈 하이퍼파라미터나 전처리에 대한 입력값들을 설정합니다. dict과 같은 hash타입의 데이터로 추가할 수 있으며 key값에는 이름이 포함되어선 안되고 value값은 10MB로 데이터크기가 제한됩니다.

*   save_code(bool, optional) : wandb를 사용하는 메인 스크립트나 노트북을 wandb 내에 저장할지를 결정합니다. default값은 False이고 설정페이지에서 default값을 변경할수도 있습니다.

*   name(str, optional) : run의 이름을 결정합니다. wandb.run.name(str)과 같은 역할을 수행합니다.

그 외에 파라미터들은 링크를 참조 부탁드립니다.







In [ ]:

#wandb.run.name = 'Baseline-Batch-64-128'

###그 외 필요한 패키지들을 import.

In [ ]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict

import torch
from torch.nn.utils.rnn import pad_sequence

import numpy as np
from tqdm import tqdm, trange

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW
)

# 1. Preprocess

In [ ]:
def make_id_file(task, tokenizer):
    """
        모델에 들어갈 데이터를 open하여 tokenizer로 encoding해준 뒤
        str형태로 저장하여 return합니다.
    """
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join(file_name), 'r', encoding='utf-8') as f:
            id_file_data = [tokenizer.encode(line.lower()) for line in f.readlines()]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    """
        이번 프로젝트의 경우 train set과 valid set data가 나누어져 제공되었기 때문에
        아래와 같이 4개로 나누어 tokenizing을 진행하였고 데이터셋이 하나인 경우
        train / valid / test 셋에 대한 비율을 6:2:2 등으로 나누어 사용
        
        ex)
        from sklearn.model_selection import train_test_split

        x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, shuffle=True)

    """
    train_pos = make_data_strings('sentiment.train.1')
    train_neg = make_data_strings('sentiment.train.0')
    dev_pos = make_data_strings('sentiment.dev.1')
    dev_neg = make_data_strings('sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

Hugging Face의 BertTokenizer중 사전학습된 'bert-base-uncased' tokenizer 호출

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
"""
colab의 기능을 이용해 colab 작업환경에 파일을 업로드하여 사용

로컬 환경에서 .py파일과 같은 작업공간에 파일을 올려두는것과 같음.
"""
uploaded = files.upload()


Saving sentiment.dev.0 to sentiment.dev.0
Saving sentiment.dev.1 to sentiment.dev.1
Saving sentiment.train.0 to sentiment.train.0
Saving sentiment.train.1 to sentiment.train.1
Saving test_no_label.csv to test_no_label.csv


In [ ]:
!ls

sample_data	 sentiment.dev.1    sentiment.train.1
sentiment.dev.0  sentiment.train.0  test_no_label.csv


In [ ]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)

it will take some times...
make id file finished!


In [ ]:
train_pos[:10]

['101 6581 2833 1012 102',
 '101 21688 8013 2326 1012 102',
 '101 2027 2036 2031 3679 19247 1998 3256 6949 2029 2003 2428 2204 1012 102',
 '101 2009 1005 1055 1037 2204 15174 2098 7570 22974 2063 1012 102',
 '101 1996 3095 2003 5379 1012 102',
 '101 2204 3347 2833 1012 102',
 '101 2204 2326 1012 102',
 '101 11350 1997 2154 2003 25628 1998 7167 1997 19247 1012 102',
 '101 2307 2173 2005 6265 2030 3347 27962 1998 5404 1012 102',
 '101 1996 2047 2846 3504 6429 1012 102']

In [ ]:
class SentimentDataset(object):
    """
        해당 프로젝트에서는 긍정과 부정 두가지에 대한 감정 이진분류 태스크이고,
        data set의 구성이 train set에 쓰일 긍정값 데이터와 부정값 데이터 하나씩과
        valid set에 쓰일 긍정값 데이터와 부정값 데이터가 하나씩 주어지므로, 용도에 
        맞게 데이터 셋을 통합하기 위한 클래스이다.
    """
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [ ]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

In [ ]:
for i, item in enumerate(train_dataset):
    print(item)
    if i == 10:
        break

(array([ 101, 6581, 2833, 1012,  102]), array([1]))
(array([  101, 21688,  8013,  2326,  1012,   102]), array([1]))
(array([  101,  2027,  2036,  2031,  3679, 19247,  1998,  3256,  6949,
        2029,  2003,  2428,  2204,  1012,   102]), array([1]))
(array([  101,  2009,  1005,  1055,  1037,  2204, 15174,  2098,  7570,
       22974,  2063,  1012,   102]), array([1]))
(array([ 101, 1996, 3095, 2003, 5379, 1012,  102]), array([1]))
(array([ 101, 2204, 3347, 2833, 1012,  102]), array([1]))
(array([ 101, 2204, 2326, 1012,  102]), array([1]))
(array([  101, 11350,  1997,  2154,  2003, 25628,  1998,  7167,  1997,
       19247,  1012,   102]), array([1]))
(array([  101,  2307,  2173,  2005,  6265,  2030,  3347, 27962,  1998,
        5404,  1012,   102]), array([1]))
(array([ 101, 1996, 2047, 2846, 3504, 6429, 1012,  102]), array([1]))
(array([ 101, 2023, 2173, 2001, 2200, 2204, 1012,  102]), array([1]))


In [ ]:
def collate_fn_style(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)
    sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids, labels

In [ ]:
train_batch_size = 64
eval_batch_size = 128

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

In [ ]:
# random seed
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
model.train()
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def compute_acc(predictions, target_labels):
    return (np.array(predictions) == np.array(target_labels)).mean()

In [ ]:
from transformers.utils.dummy_pt_objects import LogitsProcessorList
train_epoch = 3
lowest_valid_loss = 9999.

train_batch_size = 2
eval_batch_size = 512

for i in range(9, 10):
  pre_batch = pow(train_batch_size,i)
  wandb.init(
      
      project = 'each_Train_batch-512',
      entity = 'goorm_nlp_project_1',
        
      config = {
          
          'epochs': 3,
          'classses' : 2,
          'batch_size' : 512,
          'learning_rate' : 1e-5,
          'architecture' : 'BERT',
          'seed': 42
      },
      
      name = f'{pre_batch}-512',
      reinit=True
  ) 

  train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=pow(train_batch_size,i),
                                            shuffle=True, collate_fn=collate_fn_style,
                                            pin_memory=True, num_workers=2)
  dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style,
                                          num_workers=2)

  for epoch in range(train_epoch):
      
      with tqdm(train_loader, unit="batch") as tepoch:
          for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
              train_predictions = []
              train_labels = []

              tepoch.set_description(f"Epoch {epoch}")
              input_ids = input_ids.to(device)
              attention_mask = attention_mask.to(device)
              token_type_ids = token_type_ids.to(device)
              position_ids = position_ids.to(device)
              labels = labels.to(device, dtype=torch.long)

              optimizer.zero_grad()

              output = model(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            labels=labels)

              loss = output.loss
              loss.backward()

              optimizer.step()

              tepoch.set_postfix(loss=loss.item())
              logits = output.logits
              train_predictions += [0 if example[0] > example[1] else 1 for example in logits]
              train_labels += [int(example) for example in labels]
              
              
              if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                  
                  # Evaluate the model five times per epoch
                  
                  with torch.no_grad():
                      model.eval()
                      valid_losses = []
                      predictions = []
                      target_labels = []
                      for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                  desc='Eval',
                                                                                                  position=1,
                                                                                                  leave=None):
                          input_ids = input_ids.to(device)
                          attention_mask = attention_mask.to(device)
                          token_type_ids = token_type_ids.to(device)
                          position_ids = position_ids.to(device)
                          labels = labels.to(device, dtype=torch.long)

                          output = model(input_ids=input_ids,
                                        attention_mask=attention_mask,
                                        token_type_ids=token_type_ids,
                                        position_ids=position_ids,
                                        labels=labels)

                          logits = output.logits
                          loss = output.loss
                          valid_losses.append(loss.item())

                          batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                          batch_labels = [int(example) for example in labels]

                          predictions += batch_predictions
                          target_labels += batch_labels

                  acc = compute_acc(predictions, target_labels)
                  valid_loss = sum(valid_losses) / len(valid_losses)
                  train_acc = compute_acc(train_predictions, train_labels)
                  metrics={
                      'val_acc':acc,
                      'val_loss':valid_loss,
                      'train_loss': loss,
                      'train_acc' : train_acc
                  }
                  wandb.log(metrics)
                  
                  if lowest_valid_loss > valid_loss:
                      print('Acc for model which have lower valid loss: ', acc)
                      torch.save(model.state_dict(), "./pytorch_model.bin")
  wandb.finish()

train_acc,██▁
train_loss,▁▅█
val_acc,▁█▃
val_loss,▆▁█
train_acc,0.99805
train_loss,0.01543
val_acc,0.98125
val_loss,0.11932


Epoch 0:   0%|          | 0/866 [00:00<?, ?batch/s]


RuntimeError: ignored

In [ ]:
import pandas as pd
test_df = pd.read_csv('test_no_label.csv')

In [ ]:
test_dataset = test_df['Id']

In [ ]:
def make_id_file_test(tokenizer, test_dataset):
    data_strings = []
    id_file_data = [tokenizer.encode(sent.lower()) for sent in test_dataset]
    for item in id_file_data:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [ ]:
test = make_id_file_test(tokenizer, test_dataset)

In [ ]:
test[:10]

['101 2009 1005 1055 1037 2878 2047 3325 1998 2047 26389 2169 2051 2017 2175 1012 102',
 '101 2061 15640 2013 2019 2214 5440 1012 102',
 '101 2009 2003 1996 2087 14469 7273 1999 1996 3028 1012 102',
 '101 2079 2025 3696 1037 10084 2007 2122 2111 1012 102',
 '101 1045 2001 6091 1998 2016 2081 2033 2514 2061 6625 1998 6160 1012 102',
 '101 1996 2069 2518 2057 2363 2008 2001 2980 2001 1996 4157 1012 102',
 '101 2053 1010 2025 1996 3924 2012 2004 2226 1010 1996 3924 1999 3502 2152 1012 102',
 '101 2027 3288 2009 2041 2392 2005 2017 1998 2024 2200 14044 1012 102',
 '101 4606 1996 12043 2106 1050 1005 1056 2130 2113 2129 2000 2147 1996 3274 1012 102',
 '101 2027 2031 2019 6581 4989 1997 25025 2015 2000 5454 2013 1012 102']

In [ ]:
class SentimentTestDataset(object):
    def __init__(self, tokenizer, test):
        self.tokenizer = tokenizer
        self.data = []

        for sent in test:
            self.data += [self._cast_to_int(sent.strip().split())]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample)

In [ ]:
test_dataset = SentimentTestDataset(tokenizer, test)

In [ ]:
def collate_fn_style_test(samples):
    input_ids = samples
    max_len = max(len(input_id) for input_id in input_ids)

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in range(len(samples))],
                             batch_first=True)
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         range(len(samples))])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in range(len(samples))])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in range(len(samples))])

    return input_ids, attention_mask, token_type_ids, position_ids

In [ ]:
test_batch_size = 128
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions


Test: 100%|██████████| 16/16 [00:00<00:00, 20.83it/s]
                                                     

In [ ]:
test_df['Category'] = predictions

In [ ]:
test_df.to_csv('submission.csv', index=False)